### Preprocessing for the models

In [ ]:
import cv2
import numpy as np
from fastai.vision import pil2tensor, Image

def info_image(im):
    # Compute the center (cx, cy) and radius of the eye
    cy = im.shape[0]//2
    midline = im[cy,:]
    midline = np.where(midline>midline.mean()/3)[0]
    if len(midline)>im.shape[1]//2:
        x_start, x_end = np.min(midline), np.max(midline)
    else: # This actually rarely happens p~1/10000
        x_start, x_end = im.shape[1]//10, 9*im.shape[1]//10
    cx = (x_start + x_end)/2
    r = (x_end - x_start)/2
    return cx, cy, r

def resize_image(im, image_dim=None, augmentation=False):
    # Crops, resizes and potentially augments the image to image_dim
    cx, cy, r = info_image(im)
    scaling = image_dim/(2*r)
    rotation = 0
    if augmentation:
        scaling *= 1 + 0.3 * (np.random.rand()-0.5)
        rotation = 360 * np.random.rand()
    M = cv2.getRotationMatrix2D((cx,cy), rotation, scaling)
    M[0,2] -= cx - image_dim/2
    M[1,2] -= cy - image_dim/2
    # return im # disbale resizing
    return cv2.warpAffine(im,M,(image_dim,image_dim)) # This is the most important line

def crop_image_from_gray(img,tol=7):
    """
    Crop out black borders
    https://www.kaggle.com/ratthachat/aptos-updated-preprocessing-ben-s-cropping
    """  
    if img.ndim == 2: # gray scale
        mask = img>tol  # create an array (mask) of pixels higher than tolerance
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim == 3: # color image
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # convert to gray
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else: 
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

PARAM = 96
def Radius_Reduction(img,PARAM):
    h,w,c=img.shape
    Frame=np.zeros((h,w,c),dtype=np.uint8)
    cv2.circle(Frame,(int(math.floor(w/2)),int(math.floor(h/2))),int(math.floor((h*PARAM)/float(2*100))), (255,255,255), -1)
    Frame1=cv2.cvtColor(Frame, cv2.COLOR_BGR2GRAY)
    img1 =cv2.bitwise_and(img,img,mask=Frame1)
    return img
    
def contrast_and_crop(torch_tensor, image_dim=None, path=None, sigmaX=10, median_blur=True):
    if path is None:
        np_image = image2np(torch_tensor) * 255 # convert tensor image to numpy array
        np_image = np_image.astype(np.uint8)
    else:
        np_image = cv2.imread(path)  # may return a string, in which case treat it as a path
    
    image = np_image
    # image = cv2.cvtColor(np_image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (image_dim, image_dim))
    # image = resize_image(image, image_dim)
    
    # if median_blur:
    #     k = np.max(image.shape)//20*2+1
    #     blur = cv2.medianBlur(image, k)
    # else:
    #     blur = cv2.GaussianBlur( image , (0,0) , sigmaX) 
        
    # image = cv2.addWeighted ( image,4, blur,-4 ,128)

    # image = Radius_Reduction(image, PARAM)

    return pil2tensor(image, np.float32).div_(255) # return tensor

def advprop_normalise(torch_tensor):
    np_image = image2np(torch_tensor) * 2.0 - 1.0 
    return pil2tensor(np_image, np.float32) # return tensor

# we later override the load image function of fast.ai to make sure all images
# including train and test sets, are properly processed
# define function    

# def _load_format(path, convert_mode, after_open, image_dim)->Image:
#     image = contrast_and_crop(None, image_dim, path, median_blur=False) 
# 
#     return Image(image) # return fastai Image format

print("initialised preprocessing")

initialised preprocessing
